In [ ]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import torchvision

In [ ]:
dir(torch)

In [ ]:
#Initializing normalizing transform for the dataset
normalize_transform = torchvision.transforms.Compose([
	torchvision.transforms.ToTensor(),
	torchvision.transforms.Normalize(mean = (0.5, 0.5, 0.5),
									std = (0.5, 0.5, 0.5))])

#Downloading the CIFAR10 dataset into train and test sets
train_dataset = torchvision.datasets.CIFAR10(
	root="./CIFAR10/train", train=True,
	transform=normalize_transform,
	download=True)
	
test_dataset = torchvision.datasets.CIFAR10(
	root="./CIFAR10/test", train=False,
	transform=normalize_transform,
	download=True)
	
#Generating data loaders from the corresponding datasets
batch_size = 128
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [ ]:
labels_map = {
    0: "T-Shirt",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle Boot",
}

In [ ]:
batch_size = 128
train_loader = torch.utils.data.DataLoader(training_data, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)
# #Plotting 25 images from the 1st batch 
# dataiter = iter(train_loader)
# images, labels = dataiter.next()
# plt.imshow(np.transpose(torchvision.utils.make_grid(
#   images[:25], normalize=True, padding=1, nrow=5).numpy(), (1, 2, 0)))
# plt.axis('off')

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [ ]:
dir(NeuralNetwork)

In [ ]:
training_data[0][0].shape

torch.Size([1, 28, 28])

In [ ]:
#type OOP body with functional soul
class CNN(torch.nn.Module):
	def __init__(self):
		super().__init__()
		self.model = torch.nn.Sequential(
			#Input = 3 x 32 x 32, Output = 32 x 32 x 32
			torch.nn.Conv2d(in_channels = 3, out_channels = 32, kernel_size = 3, padding = 1),
			torch.nn.ReLU(),
			#Input = 32 x 32 x 32, Output = 32 x 16 x 16
			torch.nn.MaxPool2d(kernel_size=2),

			#Input = 32 x 16 x 16, Output = 64 x 16 x 16
			torch.nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3, padding = 1),
			torch.nn.ReLU(),
			#Input = 64 x 16 x 16, Output = 64 x 8 x 8
			torch.nn.MaxPool2d(kernel_size=2),
			
			#Input = 64 x 8 x 8, Output = 64 x 8 x 8
			torch.nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = 3, padding = 1),
			torch.nn.ReLU(),
			#Input = 64 x 8 x 8, Output = 64 x 4 x 4
			torch.nn.MaxPool2d(kernel_size=2),

			torch.nn.Flatten(),
			torch.nn.Linear(64*4*4, 512),
			torch.nn.ReLU(),
			torch.nn.Linear(512, 10)
		)

	def forward(self, x):
		return self.model(x)


In [ ]:
#type pure OOP
"""
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.conv2d = nn.conv2d()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
"""

In [ ]:
#Selecting the appropriate training device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = CNN().to(device)

#Defining the model hyper parameters
num_epochs = 50
learning_rate = 0.001
weight_decay = 0.01
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

#Training process begins
train_loss_list = []
for epoch in range(num_epochs):
	print(f'Epoch {epoch+1}/{num_epochs}:', end = ' ')
	train_loss = 0
	
	#Iterating over the training dataset in batches
	model.train()
	for i, (images, labels) in enumerate(train_loader):
		
		#Extracting images and target labels for the batch being iterated
		images = images.to(device)
		labels = labels.to(device)

		#Calculating the model output and the cross entropy loss
		outputs = model(images)
		loss = criterion(outputs, labels)

		#Updating weights according to calculated loss
		optimizer.zero_grad()
		loss.backward()
		optimizer.step()
		train_loss += loss.item()
	
	#Printing loss for each epoch
	train_loss_list.append(train_loss/len(train_loader))
	print(f"Training loss = {train_loss_list[-1]}")
	
#Plotting loss for all epochs
# plt.plot(range(1,num_epochs+1), train_loss_list)
# plt.xlabel("Number of epochs")
# plt.ylabel("Training loss")

Epoch 1/50: Training loss = 1.6177588506122988
Epoch 2/50: Training loss = 1.3410405408390953
Epoch 3/50: Training loss = 1.2125911372701834
Epoch 4/50: Training loss = 1.11868275538125
Epoch 5/50: Training loss = 1.0541236688718771
Epoch 6/50: Training loss = 1.009339379837446
Epoch 7/50: Training loss = 0.9826912684818668
Epoch 8/50: Training loss = 0.9622101193803656
Epoch 9/50: Training loss = 0.9439341175891555
Epoch 10/50: Training loss = 0.9293358583584466
Epoch 11/50: Training loss = 0.9180533463692726
Epoch 12/50: Training loss = 0.9074363961549061
Epoch 13/50: Training loss = 0.8957384615907888
Epoch 14/50: Training loss = 0.8863694385799301
Epoch 15/50: Training loss = 0.8780556261691901
Epoch 16/50: Training loss = 0.8701435383933279
Epoch 17/50: Training loss = 0.8624582397358497
Epoch 18/50: Training loss = 0.8566573774418258
Epoch 19/50: Training loss = 0.8503212878466262
Epoch 20/50: Training loss = 0.8447236766290787
Epoch 21/50: Training loss = 0.8379741181497988
Epoc